In [9]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
%matplotlib inline

# Baseline Model

At its most basic level, a recommendation system takes information previously collected on both users and items and is able to pair them together to predict how much a user would like new items. Often times, such a baseline is surprisingly effective, and improvements on the baseline are hard-won. 

Here, we define our baseline using a simple multiple linear regression of average rating for each user and average rating for a given restaurant, $m$, to predict what each user would rate $m$. The model is as follows:

$$\hat{Y_{um}} = \hat{\mu} + \hat{\theta_{u}} + \hat{\gamma_{m}}$$

Where $\hat{\theta_{u}}$ is the average rating for user $u$, $\hat{\gamma_{m}}$ is the average rating for restaurant $m$, and $\hat{\mu}$ is the intercept.


In our analysis, we will first run this baseline model on ratings from all 8 states of interest (NV, AZ, ON, NC, OH, PA, QC, and WI). Then we will run the model on each state separately to see how well the model predicts within market.

## Run baseline model on full universe of reviews

In [6]:
## Load in test and train data for all markets
train_df = pd.read_csv("Data/train.csv")
test_df = pd.read_csv("Data/test.csv")
print(train_df.shape)
print(test_df.shape)

(1409493, 13)
(535585, 13)


In [11]:
train_df.head()

,review_date,business_longitude,business_id,business_categories,business_name,business_state,review_score,user_id,user_average_rating,business_review_count,business_average_rating,business_latitude,user_review_count
0,2011-05-01,-115.279668,nK7JeIqdBli3umEhBIh33g,"['Italian', 'Restaurants']",Lucio Ristorante,NV,3.0,---1lKK3aKOuomHnwAkAow,3.97,31,4.0,36.143983,245
1,2013-03-11,-115.266850,S599hCA4kJJO3_b6SRFKoA,"['Restaurants', 'Mexican', 'Breakfast & Brunch...",Michoacan Gourmet Mexican Restaurant,NV,4.0,---1lKK3aKOuomHnwAkAow,3.97,324,3.5,36.271357,245
2,2010-10-17,-115.192230,2BbFeotL85cIaBjSq1SWiA,"['American (New)', 'Restaurants']",Springs Cafe by Wolfgang Puck,NV,1.0,---1lKK3aKOuomHnwAkAow,3.97,23,2.5,36.168099,245
3,2016-01-19,-115.189844,gTlDDzDEHyDQ6iwjNhpI6A,"['Restaurants', 'Indian']",Mount Everest India's Cuisine,NV,5.0,---1lKK3aKOuomHnwAkAow,3.97,1067,4.5,36.143973,245
4,2012-07-21,-115.151632,AxeQEz3-s9_1TyIo-G7UQw,"['Restaurants', 'Barbeque', 'Thai']",Thai Original BBQ Restaurant,NV,3.0,---1lKK3aKOuomHnwAkAow,3.97,87,4.0,36.153504,245


In [10]:
print(train_df.columns)
train_df.describe()

Index(['review_date', 'business_longitude', 'business_id',
       'business_categories', 'business_name', 'business_state',
       'review_score', 'user_id', 'user_average_rating',
       'business_review_count', 'business_average_rating', 'business_latitude',
       'user_review_count'],
      dtype='object')


,business_longitude,review_score,user_average_rating,business_review_count,business_average_rating,business_latitude,user_review_count
count,1.409493e+06,1.409493e+06,1.409493e+06,1.409493e+06,1.409493e+06,1.409493e+06,1.409493e+06
mean,-1.004127e+02,3.705296e+00,3.737982e+00,3.887400e+02,3.698366e+00,3.754085e+01,1.741930e+02
std,1.656028e+01,1.242746e+00,5.116403e-01,7.207097e+02,5.801115e-01,4.069533e+00,3.889996e+02
min,-1.156796e+02,1.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,3.313728e+01,0.000000e+00
25%,-1.151364e+02,3.000000e+00,3.470000e+00,6.600000e+01,3.500000e+00,3.361141e+01,2.100000e+01
50%,-1.119258e+02,4.000000e+00,3.770000e+00,1.700000e+02,4.000000e+00,3.611304e+01,5.600000e+01
75%,-8.008083e+01,5.000000e+00,4.060000e+00,3.950000e+02,4.000000e+00,4.148515e+01,1.720000e+02
max,1.150868e+02,5.000000e+00,5.000000e+00,6.979000e+03,5.000000e+00,4.834344e+01,1.106500e+04


In [13]:
# Define X as user average and business average, Y as given rating for test and train data
X_train_all = train_df[['user_average_rating', 'business_average_rating']]
y_train_all = train_df['review_score']
X_test_all = test_df[['user_average_rating', 'business_average_rating']]
y_test_all = test_df['review_score']

In [17]:
# Fit linear regression using training data and test using the testing data
baseline_all = LinearRegression(fit_intercept=True)
baseline_all.fit(X_train_all, y_train_all)

print('Baseline Intercept:', baseline_all.intercept_)
print('Baseline Coefficients:', baseline_all.coef_)
print('Baseline Train Score:', baseline_all.score(X_train_all, y_train_all))
print('Baseline Test Score:', baseline_all.score(X_test_all, y_test_all))

Baseline Intercept: -2.0590063094
Baseline Coefficients: [ 0.8109637  0.7389574]
Baseline Train Score: 0.266157747505
Baseline Test Score: 0.272368710332


## Run baseline model for each of the eight markets